In [180]:
import pandas as pd
import numpy as np

In [181]:
import json
from pandas.io.json import json_normalize

In [182]:
df = pd.read_json('data/world_bank_projects.json')

## 1. Find the 10 countries with most projects

In [183]:
df.countryname.value_counts(dropna=False).head(10)

People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Nepal                              12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

## 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [184]:
json_string = json.load((open('data/world_bank_projects.json')))
theme_df = json_normalize(json_string, 'mjtheme_namecode')

In [185]:
theme_df.name.value_counts(dropna=False).head(10)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

## 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [186]:
theme_df.dtypes
# converting unicode to int and str for code and name resp.
theme_df.code = theme_df.code.astype(np.int64)
theme_df.name = theme_df.name.astype(np.str)
theme_df.dtypes
type(theme_df.name[0])

str

In [187]:
# convert empty entries to NaN
theme_df.name = theme_df.name.apply(lambda x: x.strip()).replace('', np.nan)

In [188]:
theme_df.isnull().sum()

code      0
name    122
dtype: int64

In [189]:
# create a lookup table
lookup_df = theme_df.copy()

In [190]:
lookup_df.isnull().sum()

code      0
name    122
dtype: int64

In [191]:
# sanitize
lookup_df = lookup_df.dropna(axis=0)

In [192]:
lookup_df = lookup_df.drop_duplicates()

In [193]:
# create series out of the lookup df
lookup_series = lookup_df.set_index('code').name

In [194]:
theme_df.isnull().sum()


code      0
name    122
dtype: int64

In [195]:
theme_df.loc[theme_df['name'].isnull(),'name'] = theme_df['code'].map(lookup_series)

In [196]:
theme_df.isnull().sum()

code    0
name    0
dtype: int64

In [197]:
theme_df.name.value_counts(dropna=False).head(10)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64